In [1]:
import torch
import torchvision.transforms as transforms
from model import CSRNet

In [2]:
# Load the model
model = CSRNet()
model = model.cuda()
checkpoint = torch.load('partBmodel_best.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
model.eval()

/home/omar/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/omar/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CSRNet(
  (frontend): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

In [3]:
# Define transformer to preprocess the input image
transform = transforms.Compose([
    transforms.ToTensor(),            # Convert to tensor
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],   # Normalize pixel values
        std=[0.229, 0.224, 0.225]
    )
])

In [5]:
import cv2


# Capture the video
cap = cv2.VideoCapture(
    './ext_testing/videos/istockphoto-1424184696-640_adpp_is.mp4')

if (cap.isOpened() == False):
    print("Error opening video")

In [6]:
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        # Preprocess the input frame
        input_frame = transform(frame).unsqueeze(0).cuda()

        # Run the input through the model
        with torch.no_grad():
            output = model(input_frame)

        # Retrieve the output
        count = output.cpu().sum().item()
        countText = f"Count : {int(count)}"

        cv2.putText(frame, countText, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 50), 2, cv2.LINE_AA)

        # Display the frame
        cv2.imshow("frame", frame)

        # Quit when q is pressed
        if cv2.waitKey(1) == ord('q'):
            break

    else:
        break

# Release the capture object and destroy the window
cap.release()
cv2.destroyAllWindows()